# Importing Packages 

In [2]:
#============ Importing Packages ============# 

#--------- Drawing Packages ---------#

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator, NullFormatter, LogLocator)
from set_size import set_size
from collections import Counter

#--------- Tensorflow Packages ---------#
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import f1_score
from sklearn.utils import class_weight
from tensorflow.keras.metrics import Metric
from sklearn.metrics import balanced_accuracy_score

#============== Packages for word2vec ==============#
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#============== Packages for classification ==============#
from sklearn.linear_model import LinearRegression, PoissonRegressor
from sklearn.preprocessing import normalize
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.svm import SVC

#--------- Utilities Packages ---------#

import sys
print(sys.executable)
import os
import re
import pdb
import shelve
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, hstack
from tqdm import tqdm
from sklearn.utils import class_weight
import enchant

import nltk
import obspy

#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('word_tokenize')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.corpus import words as dict_w
from nltk.stem.porter import PorterStemmer

# Scipy Signal
from scipy import signal

# Detrend the Signal
from obspy.signal.detrend import polynomial

#--------- Remove Warnings ---------#
import warnings
warnings.filterwarnings("ignore")


/home/chiangwe/anaconda3/envs/NetHawkes/bin/python


In [3]:
#========= Read in =========#
df = pd.read_csv('Eluvio_DS_Challenge_processes.csv')
#display( df.sort_values('up_votes', ascending=False).head(5)['title'].values )

class_weights = class_weight.compute_class_weight('balanced',  np.unique(df['label']), df['label'])
class_weights = dict(zip( np.unique(df['label']), class_weights))


df = df[ df['title_clean'].apply(lambda x: type(x)==str) ] 
y_true = df['label']


In [13]:
def create_model(): 
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(30, activation='relu'))
    model.add(tf.keras.layers.Dense(20, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='relu'))
    model.add(tf.keras.layers.Dense(5, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.BinaryAccuracy(name='binary_accuracy')])
    
    return model

embed = hub.load("https://tfhub.dev/google/nnlm-en-dim50/2")
X_train_in = embed(df['title'].values).numpy()

pairs_true_pred = []
for each_seed in [42, 50, 123]:
    
    X_train, X_test, y_train, y_test = \
        train_test_split(X_train_in, y_true, test_size=0.33, random_state=each_seed)
    
    # Create a basic model instance
    model = create_model()

    # Display the model's architecture
    #model.summary()
    
    # Callback define
    patience = 3; epochs = 70;
    checkpoint_filepath = './check_point/01_sim_tdidfNN_mdl.ckpt';

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='max')

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=False, monitor='val_loss', mode='max', save_best_only=True)

    # fit the model
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), \
                    epochs=epochs, batch_size=36, verbose=2, class_weight=class_weights,\
                    callbacks=[early_stopping, model_checkpoint_callback])

    # Load model and evaluate on test
    model = tf.keras.models.load_model('./check_point/01_sim_tdidfNN_mdl.ckpt')
    pred_test = model.predict(X_test) > 0.5;
    
    pairs_true_pred.append([y_test, pred_test])


Epoch 1/70
INFO:tensorflow:Assets written to: ./check_point/01_sim_tdidfNN_mdl.ckpt/assets


INFO:tensorflow:Assets written to: ./check_point/01_sim_tdidfNN_mdl.ckpt/assets


9394/9394 - 18s - loss: 0.6642 - binary_accuracy: 0.6194 - val_loss: 0.7127 - val_binary_accuracy: 0.5395
Epoch 2/70
9394/9394 - 17s - loss: 0.6571 - binary_accuracy: 0.6274 - val_loss: 0.6900 - val_binary_accuracy: 0.5851
Epoch 3/70
9394/9394 - 17s - loss: 0.6543 - binary_accuracy: 0.6257 - val_loss: 0.6566 - val_binary_accuracy: 0.6177
Epoch 4/70
9394/9394 - 19s - loss: 0.6526 - binary_accuracy: 0.6267 - val_loss: 0.6384 - val_binary_accuracy: 0.6540
Epoch 1/70
INFO:tensorflow:Assets written to: ./check_point/01_sim_tdidfNN_mdl.ckpt/assets


INFO:tensorflow:Assets written to: ./check_point/01_sim_tdidfNN_mdl.ckpt/assets


9394/9394 - 18s - loss: 0.6635 - binary_accuracy: 0.6254 - val_loss: 0.6836 - val_binary_accuracy: 0.5869
Epoch 2/70
INFO:tensorflow:Assets written to: ./check_point/01_sim_tdidfNN_mdl.ckpt/assets


...

9394/9394 - 18s - loss: 0.6566 - binary_accuracy: 0.6339 - val_loss: 0.7043 - val_binary_accuracy: 0.5687
Epoch 3/70
9394/9394 - 18s - loss: 0.6543 - binary_accuracy: 0.6376 - val_loss: 0.6825 - val_binary_accuracy: 0.5952
Epoch 4/70
9394/9394 - 17s - loss: 0.6524 - binary_accuracy: 0.6338 - val_loss: 0.6359 - val_binary_accuracy: 0.6688
Epoch 5/70
9394/9394 - 18s - loss: 0.6512 - binary_accuracy: 0.6417 - val_loss: 0.6576 - val_binary_accuracy: 0.6446
Epoch 1/70
INFO:tensorflow:Assets written to: ./check_point/01_sim_tdidfNN_mdl.ckpt/assets


INFO:tensorflow:Assets written to: ./check_point/01_sim_tdidfNN_mdl.ckpt/assets


9394/9394 - 18s - loss: 0.6657 - binary_accuracy: 0.6209 - val_loss: 0.6664 - val_binary_accuracy: 0.6132
Epoch 2/70
9394/9394 - 19s - loss: 0.6591 - binary_accuracy: 0.6227 - val_loss: 0.6374 - val_binary_accuracy: 0.6586
Epoch 3/70
INFO:tensorflow:Assets written to: ./check_point/01_sim_tdidfNN_mdl.ckpt/assets


INFO:tensorflow:Assets written to: ./check_point/01_sim_tdidfNN_mdl.ckpt/assets


9394/9394 - 19s - loss: 0.6568 - binary_accuracy: 0.6228 - val_loss: 0.6880 - val_binary_accuracy: 0.5727
Epoch 4/70
9394/9394 - 18s - loss: 0.6550 - binary_accuracy: 0.6252 - val_loss: 0.6686 - val_binary_accuracy: 0.5981
Epoch 5/70
9394/9394 - 17s - loss: 0.6535 - binary_accuracy: 0.6206 - val_loss: 0.6800 - val_binary_accuracy: 0.5581
Epoch 6/70
9394/9394 - 17s - loss: 0.6526 - binary_accuracy: 0.6202 - val_loss: 0.6460 - val_binary_accuracy: 0.6299


In [14]:
all_true_pred = np.hstack([ np.hstack([np.expand_dims(each[0], 1), each[1]]) for each in pairs_true_pred]).T
print(all_true_pred.shape)
np.save('SGD_NN_PreTrain.npy', all_true_pred)

(6, 166557)


In [15]:
from sklearn.metrics import recall_score, precision_score, f1_score, precision_recall_curve, precision_recall_curve
print(all_true_pred.shape)
baccu=[]; recall = []; prec =[]; f1=[]; 
for each in range(0, 3):
    baccu.append( balanced_accuracy_score(all_true_pred[2*each,:], all_true_pred[2*each+1,:]) )
    recall.append( recall_score(all_true_pred[2*each,:], all_true_pred[2*each+1,:], average='binary') )
    prec.append( precision_score(all_true_pred[2*each,:], all_true_pred[2*each+1,:], average='binary') )
    f1.append( f1_score(all_true_pred[2*each,:], all_true_pred[2*each+1,:], average='binary') )
    
# Use AUC function to calculate the area under the curve of precision recall curve
print(baccu)
print("baccu: ", np.mean(baccu))
print("recall: ", np.mean(recall))
print("prec: ", np.mean(prec))
print("f1: ", np.mean(f1))


(6, 166557)
[0.60274534621625397, 0.60867233773125151, 0.61077751693476734]
baccu:  0.607398400294
recall:  0.670220659037
prec:  0.173767957342
f1:  0.275926942666


In [ ]:
#== Test
from sklearn.decomposition import IncrementalPCA

each_seed = 42; chi_score= ls_chi_score[0];
X_train, X_test, y_train, y_test,tol_list_tr, tol_list_te = \
    train_test_split(X_tfidf, y_true, d_index, test_size=0.33, random_state=each_seed)

top_k =np.round(X_train.shape[1]/2).astype(int)
skb = SelectKBest(chi2, k=top_k ).fit(X_train, y_train)
X_train = skb.transform(X_train)
X_test = skb.transform(X_test)
#=========== Google Trend =============#
chi_score_index = np.argsort(chi_score)[::-1]
print(type(X_tfidf_google))
X_g_train = X_tfidf_google[ np.array(tol_list_tr).T, :]
X_g_test = X_tfidf_google[ np.array(tol_list_te).T, :]
    
pca = PCA(n_components=30)
pcscs = pca.fit(X_train.toarray())
X_tfidf_pca = pcscs.transform(X_train.toarray())
X_test_pca = pcscs.transform(X_test.toarray())

pca = IncrementalPCA(n_components=30)
pcscs = pca.fit(X_g_train)
X_g_train_pca = pcscs.transform(X_g_train)
X_g_test_pca = pcscs.transform(X_g_test)

X_train = np.concatenate((X_tfidf_pca, X_g_train, X_g_train_pca), axis=1)
X_test = np.concatenate(( X_test_pca, X_g_test, X_g_test_pca), axis=1)

In [ ]:
print(X_tfidf[1000:1010, 0:400])

In [ ]:
def create_model(input_shape):
    model = Sequential()
    model.add(Dense(100, activation='relu', kernel_initializer=tf.keras.initializers.Zeros(), input_shape=(input_shape,)))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation= 'relu', kernel_initializer=tf.keras.initializers.Zeros()))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation= 'relu', kernel_initializer=tf.keras.initializers.Zeros()))
    model.add(Dense(1, activation='sigmoid'))

    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.BinaryAccuracy(name='binary_accuracy')])
    
    return model

In [ ]:
# Create a basic model instance
model = create_model(X_train.shape[1])

# Display the model's architecture
#model.summary()

# Callback define
patience = 3; epochs = 70;
checkpoint_filepath = './check_point/01_sim_tdidfNN_mdl.ckpt';

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='max')

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False, monitor='val_loss', mode='max', save_best_only=True)

# fit the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), \
                epochs=epochs, batch_size=36, verbose=2, class_weight=class_weights,\
                callbacks=[early_stopping, model_checkpoint_callback])

# Load model and evaluate on test
model = tf.keras.models.load_model('./check_point/01_sim_tdidfNN_mdl.ckpt')
pred_test = model.predict(X_test) > 0.5;

#pairs_true_pred.append([y_test, pred_test])

In [ ]:
balanced_accuracy_score(y_test, pred_test)

In [ ]:
# Define a simple sequential model
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.decomposition import PCA

pairs_true_pred = []
for each_seed, chi_score in zip([42, 50, 123], ls_chi_score):
    X_train, X_test, y_train, y_test,tol_list_tr, tol_list_te = \
        train_test_split(X_tfidf, y_true, d_index, test_size=0.33, random_state=each_seed)
    
    top_k =np.round(X_train.shape[1]/4).astype(int)
    skb = SelectKBest(chi2, k=top_k ).fit(X_train, y_train)
    X_train = skb.transform(X_train)
    X_test = skb.transform(X_test)
    
    #=========== Google Trend =============#
    chi_score_index = np.argsort(chi_score)[::-1]
    X_g_train = X_tfidf_google.toarray()[:, chi_score_index[0:top_k] ][ np.array(tol_list_tr), :]
    X_g_test = X_tfidf_google.toarray()[:, chi_score_index[0:top_k] ][ np.array(tol_list_te), :]
        
    pca = PCA(n_components=10)
    pcscs = pca.fit(X_train.toarray())
    X_tfidf_pca = pcscs.transform(X_train.toarray())
    X_test_pca = pcscs.transform(X_test.toarray())
    
    pcscs = pca.fit(X_g_train)
    X_g_train_pca = pcscs.transform(X_g_train)
    X_g_test_pca = pcscs.transform(X_g_test)
    
    X_train = np.concatenate((X_train.toarray(), X_tfidf_pca, X_g_train, X_g_train_pca), axis=1)
    
    X_test = np.concatenate((X_test.toarray(), X_test_pca, X_g_test, X_g_test_pca), axis=1)
    

    # Create a basic model instance
    model = create_model(X_train.shape[1])

    # Display the model's architecture
    #model.summary()
    
    # Callback define
    patience = 3; epochs = 70;
    checkpoint_filepath = './check_point/01_sim_tdidfNN_mdl.ckpt';

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='max')

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=False, monitor='val_loss', mode='max', save_best_only=True)

    # fit the model
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), \
                    epochs=epochs, batch_size=36, verbose=2, class_weight=class_weights,\
                    callbacks=[early_stopping, model_checkpoint_callback])

    # Load model and evaluate on test
    model = tf.keras.models.load_model('./check_point/01_sim_tdidfNN_mdl.ckpt')
    pred_test = model.predict(X_test) > 0.5;
    
    pairs_true_pred.append([y_test, pred_test])

In [ ]:

all_true_pred = np.hstack([ np.hstack([np.expand_dims(each[0], 1), each[1]]) for each in pairs_true_pred]).T
print(all_true_pred.shape)
np.save('SGD_NN_PCA_google.npy', all_true_pred)

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score, precision_recall_curve, precision_recall_curve
print(all_true_pred.shape)
baccu=[]; recall = []; prec =[]; f1=[]; 
for each in range(0, 3):
    baccu.append( balanced_accuracy_score(all_true_pred[2*each,:], all_true_pred[2*each+1,:]) )
    recall.append( recall_score(all_true_pred[2*each,:], all_true_pred[2*each+1,:], average='binary') )
    prec.append( precision_score(all_true_pred[2*each,:], all_true_pred[2*each+1,:], average='binary') )
    f1.append( f1_score(all_true_pred[2*each,:], all_true_pred[2*each+1,:], average='binary') )
    
# Use AUC function to calculate the area under the curve of precision recall curve
print(baccu)
print("baccu: ", np.mean(baccu))
print("recall: ", np.mean(recall))
print("prec: ", np.mean(prec))
print("f1: ", np.mean(f1))


In [ ]:
all_true_pred = np.vstack([ np.vstack(each) for each in pairs_true_pred])
np.save('SGD_PCA.npy', all_true_pred)

# ============== 

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score, precision_recall_curve, precision_recall_curve

baccu=[]; recall = []; prec =[]; f1=[]; 
for each in range(0, 3):
    baccu.append( balanced_accuracy_score(all_true_pred[2*each,:], all_true_pred[2*each+1,:]) )
    recall.append( recall_score(all_true_pred[2*each,:], all_true_pred[2*each+1,:], average='binary') )
    prec.append( precision_score(all_true_pred[2*each,:], all_true_pred[2*each+1,:], average='binary') )
    f1.append( f1_score(all_true_pred[2*each,:], all_true_pred[2*each+1,:], average='binary') )
    
# Use AUC function to calculate the area under the curve of precision recall curve
print("baccu: ", np.mean(baccu))
print("recall: ", np.mean(recall))
print("prec: ", np.mean(prec))
print("f1: ", np.mean(f1))


In [ ]:
#========= Median positive and negative =========#

X_train, X_test, y_train, y_test, weight_train, weight_test = \
    train_test_split(X_new, y_true, sample_weight, test_size=0.33, random_state=42)

print( np.array([type(each)!=bool for each in y_true]).sum()  ) 
#class_weights = class_weight.compute_class_weight('balanced',
#                                                 np.unique(y_train),
#                                                 y_train)
#class_weights = dict(zip( np.unique(y_train), class_weights))
#class_weights[True] = class_weights[True]*1.0
##
## Let's do sample weights
#min_pos = df[ df['label'] == True]['up_votes'].min()
#max_neg = df[ df['label'] == False]['up_votes'].max()
#
#
#
#print(dict(zip( np.unique(y_train), class_weights)) )
#


In [ ]:

# Define a simple sequential model
def create_model():
    model = Sequential()
    model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(X_train.shape[1],)))
    model.add(Dense(2, activation= 'relu', kernel_initializer='he_normal'))
    model.add(Dense(1, activation='sigmoid'))

    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.BinaryAccuracy(name='binary_accuracy')])
    
    return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()


In [ ]:
# Callback define
patience = 3; epochs = 70;
checkpoint_filepath = './check_point/01_sim_tdidfNN_mdl.ckpt';

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='max')

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False, monitor='val_loss', mode='max', save_best_only=True)

# fit the model
history = model.fit(X_train.toarray(), y_train, validation_data=(X_test.toarray(), y_test), \
                    epochs=epochs, batch_size=36, verbose=2, class_weight=class_weights,\
                    callbacks=[early_stopping, model_checkpoint_callback])


In [ ]:
# Load model and evaluate on test
model = tf.keras.models.load_model('./check_point/01_sim_tdidfNN_mdl.ckpt')
pred_test = model.predict(X_test.toarray()) > 0.5;

from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score
print( "accuracy_score: ", accuracy_score(y_test, pred_test) )
print( "balanced_accuracy_score: ", balanced_accuracy_score(y_test, pred_test) )

tn, fp, fn, tp = confusion_matrix(y_test, pred_test).ravel()
print(tn, fp, fn, tp)
print(pred_test.shape)

In [ ]:
### Try Google trends

In [ ]:
# Define a sequential model
# Use Token based text embedding trained on English Google News 7B corpus
def create_model(): 
    
    embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
    hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
    model = tf.keras.Sequential()
    model.add(hub_layer)
    model.add(tf.keras.layers.Dense(10, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.BinaryAccuracy(name='binary_accuracy')])
    
    return model
    
# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()


In [ ]:
# Train and Test seprate
X_train, X_test, y_train, y_test = train_test_split(df['title_clean'], y_true, test_size=0.33, random_state=42)

# Callback define
patience = 3; epochs = 70;
checkpoint_filepath = './check_point/02_pre_nnlm-en-dim50_mdl.ckpt';

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='max')

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False, monitor='val_loss', mode='max', save_best_only=True)


# fit the model
history = model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), \
                    epochs=epochs, batch_size=64, verbose=2, sample_weight=weight_train,\
                    callbacks=[early_stopping, model_checkpoint_callback])


In [ ]:
# Load model and evaluate on test
model = tf.keras.models.load_model('./check_point/02_pre_nnlm-en-dim50_mdl.ckpt')
pred_test = model.predict(X_test) > 0.5;

from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score
print( "accuracy_score: ", accuracy_score(y_test, pred_test) )
print( "balanced_accuracy_score: ", balanced_accuracy_score(y_test, pred_test) )

tn, fp, fn, tp = confusion_matrix(y_test, pred_test).ravel()
print(tn, fp, fn, tp)
print(pred_test.shape)

In [ ]:
# Define a sequential model
# Use Token based text embedding trained on English Google News 7B corpus
# Use pretrain embedding

embed = hub.load("https://tfhub.dev/google/nnlm-en-dim50/2")
X_train = embed(df['title_clean'].values).numpy()

# Train and Test seprate
X_train, X_test, y_train, y_test = train_test_split(X_train, y_true, test_size=0.33, random_state=42)

# Define a simple sequential model
def create_model():
    model = Sequential()
    model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(X_train.shape[1],)))
    model.add(Dense( 2, activation= 'relu', kernel_initializer='he_normal'))
    model.add(Dense( 1, activation='sigmoid'))

    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.BinaryAccuracy(name='binary_accuracy')])
    
    return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()



In [ ]:

# Callback define
patience = 3; epochs = 70;
checkpoint_filepath = './check_point/03_preEmbed_nnlm-en-dim50_mdl.ckpt';

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='max')

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False, monitor='val_loss', mode='max', save_best_only=True)


# fit the model
history = model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), \
                    epochs=epochs, batch_size=64, verbose=2, sample_weight=weight_train,\
                    callbacks=[early_stopping, model_checkpoint_callback])


In [ ]:
# Load model and evaluate on test
model = tf.keras.models.load_model('./check_point/03_preEmbed_nnlm-en-dim50_mdl.ckpt')
pred_test = model.predict(X_test) > 0.5;

from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score
print( "accuracy_score: ", accuracy_score(y_test, pred_test) )
print( "balanced_accuracy_score: ", balanced_accuracy_score(y_test, pred_test) )

tn, fp, fn, tp = confusion_matrix(y_test, pred_test).ravel()
print(tn, fp, fn, tp)
print(pred_test.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['title_clean'], y_true, test_size=0.33, random_state=42)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x=X_train, y=y_train,
                    epochs=150, batch_size=32, verbose=2, class_weight=class_weights,
                    validation_data=(X_test, y_test))

In [ ]:
reg.score(X_tfidf, df['up_votes'])

In [ ]:
reg.predict(X_tfidf).mean()


In [ ]:
display(X_tfidf.sum(0).mean() )
display(df['up_votes'].mean() )

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
#regr = MLPRegressor(random_state=1, max_iter=500).fit(X_tfidf, df['up_votes'].values)

In [ ]:
print((df['up_votes'].values > df['up_votes'].values.mean()).sum())
print(df.shape)

In [ ]:
y_true = (df['up_votes'].values>np.quantile( df['up_votes'].values, 0.50))

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_tfidf, y_true )

In [ ]:
y_pred = clf.predict(X_tfidf)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_true, y_pred))

In [ ]:
'''
# Customize my own metrics

class BalancedAccuracy(Metric):
    def __init__(self, name="balanced_accuracy", **kwargs):
        super(BalancedAccuracy, self).__init__(name=name, **kwargs)
        self.balanced_accuracy = self.add_weight(name="ctp", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = y_pred.nupmy()
        y_true = y_true.nupmy()

        value = balanced_accuracy_score(y_true, y_pred, sample_weight)
        #values = tf.multiply(values, sample_weight)
        self.balanced_accuracy.assign_add((value))

    def result(self):
        return self.balanced_accuracy

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.balanced_accuracy.assign(0.0)
'''